# Benchmark Analysis on Bayesian Logistic Regressions

The following notebooks makes use of the following UCI datasets:

1. Wine
2. Rice
3. Diabetes
4. Covertype

In [1]:
library(dplyr)
library(gridExtra)
library(ggplotify)
library(ggplot2)
library(foreach)
library(kableExtra)

source("logitregress.R")


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'gridExtra'


The following object is masked from 'package:dplyr':

    combine



Attaching package: 'kableExtra'


The following object is masked from 'package:dplyr':

    group_rows


Loading required package: mvtnorm

Loading required package: randtoolbox

Loading required package: rngWELL

This is randtoolbox. For an overview, type 'help("randtoolbox")'.

Loading required package: distr

Loading required package: startupmsg

Utilities for Start-Up Messages (version 0.9.6.1)

For more information see ?"startupmsg", NEWS("startupmsg")


Loading required package: sfsmisc


Attaching package: 'sfsmisc'


The following object is masked from 'package:dplyr':

    last


Object Oriented Implementation of Distributions (version 2.9.3)

Attention: Arithmetics on distribut

In [25]:
source("logitregress.R")
wine.rbl <- bbvi_logitreg(categories = wine.train[,1], X = scale(wine.train[,-1]), mc_size = 100, method = "RB", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = 9, verbose = TRUE)


BBVI-RB: Iteration 100 | lambda: 0.0289 | ELBO: -557 | ELBO Change: 0

BBVI-RB: Iteration 200 | lambda: 0.017 | ELBO: -557.35 | ELBO Change: 0

BBVI-RB: Iteration 300 | lambda: 0.0251 | ELBO: -523.68 | ELBO Change: 0

BBVI-RB: Iteration 400 | lambda: 0.0098 | ELBO: -497.03 | ELBO Change: 0

BBVI-RB: Iteration 500 | lambda: 0.0241 | ELBO: -573.39 | ELBO Change: 0

BBVI-RB: Iteration 600 | lambda: 0.009 | ELBO: -534.82 | ELBO Change: 0

BBVI-RB: Iteration 700 | lambda: 0.019 | ELBO: -561.62 | ELBO Change: 0

BBVI-RB: Iteration 800 | lambda: 0.0075 | ELBO: -531.75 | ELBO Change: 0

BBVI-RB: Iteration 900 | lambda: 0.0051 | ELBO: -523.32 | ELBO Change: 0

BBVI-RB: Iteration 1000 | lambda: 0.0043 | ELBO: -456.6 | ELBO Change: 0

Algorithm did not converge after 1000 steps. Results may not be reliable.



## Wine

From the UCI Library: https://archive.ics.uci.edu/dataset/109/wine

In [2]:
wine <- read.table("Data/wine.csv", sep=",", header = FALSE)
wine <- wine[wine$V1 %in% c(1,3), ]
wine[,1] <- sapply(wine[[1]], function(x){
    if(x == 3) return(0)
    return(1)
})

set.seed(923)
index <- sample(x = nrow(wine), size = round(0.7 * nrow(wine)))

wine.train <- as.matrix(wine[index, ])
wine.test <- as.matrix(wine[-index, ])

rownames(wine.train) <- NULL
rownames(wine.test) <- NULL

In [17]:
wine_performances <- foreach(seed = 0:9) %do%{
    wine.nav <- bbvi_logitreg(categories = wine.train[,1], X = scale(wine.train[,-1]), mc_size = 100, method = "Naive", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    wine.rbl <- bbvi_logitreg(categories = wine.train[,1], X = scale(wine.train[,-1]), mc_size = 100, method = "RB", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    wine.jsp <- bbvi_logitreg(categories = wine.train[,1], X = scale(wine.train[,-1]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    wine.rbp <- bbvi_logitreg(categories = wine.train[,1], X = scale(wine.train[,-1]), mc_size = 100, method = "RB+", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)

    wine.nav.acc <- logitreg_accuracy(wine.nav, newdata = list(categories=wine.test[,1], X=scale(wine.test[,-1])))
    wine.rbl.acc <- logitreg_accuracy(wine.rbl, newdata = list(categories=wine.test[,1], X=scale(wine.test[,-1])))
    wine.jsp.acc <- logitreg_accuracy(wine.jsp, newdata = list(categories=wine.test[,1], X=scale(wine.test[,-1])))
    wine.rbp.acc <- logitreg_accuracy(wine.rbp, newdata = list(categories=wine.test[,1], X=scale(wine.test[,-1])))

    return(list(
        "Summary" = rbind(
            cbind(Method = "Naive", logitreg_summary(wine.nav)),
            cbind(Method = "RB", logitreg_summary(wine.rbl)),
            cbind(Method = "JS+", logitreg_summary(wine.jsp)),
            cbind(Method = "RB+", logitreg_summary(wine.rbp))
        ),
        "Accuracy" = rbind(
            cbind(Method="Naive", AUC=wine.nav.acc$auc_postmean),
            cbind(Method="RB", AUC=wine.rbl.acc$auc_postmean),
            cbind(Method="JS+", AUC=wine.jsp.acc$auc_postmean),
            cbind(Method="RB+", AUC=wine.rbp.acc$auc_postmean)
        )))
}

Algorithm converged after 63 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0018 | ELBO: -334.42 | ELBO Change: 0

BBVI-Naive: Iteration 200 | lambda: 0.0018 | ELBO: -605.18 | ELBO Change: 0

Algorithm converged after 206 iterations.

Algorithm converged after 76 iterations.

Algorithm converged after 70 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0029 | ELBO: -546.55 | ELBO Change: 0

Algorithm converged after 115 iterations.

Algorithm converged after 76 iterations.

Algorithm converged after 59 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0026 | ELBO: -384.72 | ELBO Change: 0

Algorithm converged after 176 iterations.

Algorithm converged after 58 iterations.

Algorithm converged after 65 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0038 | ELBO: -598.07 | ELBO Change: 0

Algorithm converged after 195 iterations.

YOASOVI-Naive: Iteration 100 | ELBO: -8.42

Algorithm converged after 115 iterations.

Algorithm converged after 36 iterations.

BBVI-Naive: Iteratio

Method,iterations_mn,iterations_sd,time_mn,time_sd,elbo_mn,elbo_sd,DIC_mn,DIC_sd
<chr>,<dbl>,<dbl>,<drtn>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BBVI,57.5,24.51,3.55 secs,1.56,-95.59,21.34,255.05,42.73
QMCVI,183.5,28.81,2.16 secs,0.22,-332.66,59.91,700.68,117.42
YOASOVI,78.9,19.34,0.04 secs,0.02,-5.44,1.77,72.76,4.78


In [6]:
wine_performances <- do.call('rbind', wine_performances)
wine_performances <- wine_performances %>%
    group_by(Method) %>%
    summarise(iterations_mn = round(mean(iter),2), iterations_sd = round(sd(iter),2), time_mn = round(mean(elapsed),2), time_sd = round(sd(elapsed),2),
                elbo_mn = round(mean(elbo),2), elbo_sd = round(sd(elbo),2), DIC_mn = round(mean(DIC),2), DIC_sd = round(sd(DIC),2))

wine_performances

BBVI-Naive: Iteration 100 | lambda: 0.0034 | ELBO: -544.52 | ELBO Change: 0

Algorithm did not converge after 100 steps. Results may not be reliable.



In [13]:
wine.bbvi.acc <- logitreg_accuracy(wine.bbvi, newdata = list(categories=wine.test[,1], X=scale(wine.test[,-1])))


Method,AUC
Naive,1


In [15]:
wine_accuracy <- round(100 * rbind(
    wine.bbvi.acc$auc_simulations,
    wine.qmcvi.acc$auc_simulations,
    wine.yoasovi.acc$auc_simulations
),2)

wine_accuracy

mean,sd
84.71,14.92
78.54,16.84
80.78,16.66


### Fit Performance

## Rice

From the UCI Library: https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik

### Predictive Performance

In [18]:
rice <- read.table("Data/rice.csv", sep=",", header = TRUE)
rice$X <- NULL
rice$Class <- sapply(rice$Class, function(x){
    if(x == "Cammeo") return(1)
    return(0)
})

set.seed(923)
index <- sample(x = nrow(rice), size = round(0.8 * nrow(rice)))

rice.train <- as.matrix(rice[index, c(8, 1:7)])
rice.test <- as.matrix(rice[-index, c(8, 1:7)])

In [19]:
rice.bbvi <- bbvi_logitreg(categories = rice.train[,1], X = scale(rice.train[,-1]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 500, converge = 1e-3, verbose = TRUE)
rice.qmcvi <- qmcvi_logitreg(categories = rice.train[,1], X = scale(rice.train[,-1]), mc_size = 10, learn_rate = rate_adagrad(1e-1), max_iter = 500, converge = 1e-3, verbose = TRUE)
rice.yoasovi <- yoasovi_logitreg(categories = rice.train[,1], X = scale(rice.train[,-1]), learn_rate = rate_adagrad(1e-1), max_iter = 1000, patience=50, verbose = TRUE)

Algorithm converged after 75 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0028 | ELBO: -18603.19 | ELBO Change: 0

Algorithm converged after 178 iterations.

Algorithm converged after 94 iterations.



In [20]:
rice.bbvi.acc <- logitreg_accuracy(rice.bbvi, newdata = list(categories=rice.test[,1], X=scale(rice.test[,-1])))
rice.qmcvi.acc <- logitreg_accuracy(rice.qmcvi, newdata = list(categories=rice.test[,1], X=scale(rice.test[,-1])))
rice.yoasovi.acc <- logitreg_accuracy(rice.yoasovi, newdata = list(categories=rice.test[,1], X=scale(rice.test[,-1])))

In [21]:
rice_accuracy <- round(100 * rbind(
    rice.bbvi.acc$auc_simulations,
    rice.qmcvi.acc$auc_simulations,
    rice.yoasovi.acc$auc_simulations
),2)

rice_accuracy

mean,sd
84.92,12.95
74.71,15.35
81.01,14.40


### Fit Performance

In [22]:
rice_performances <- foreach(seed = 0:9) %do%{
    rice.bbvi <- bbvi_logitreg(categories = rice.train[,1], X = scale(rice.train[,-1]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 500, converge = 1e-3, seed = seed, verbose = TRUE)
    rice.qmcvi <- qmcvi_logitreg(categories = rice.train[,1], X = scale(rice.train[,-1]), mc_size = 10, learn_rate = rate_adagrad(1e-1), max_iter = 500, converge = 1e-3, seed = seed, verbose = TRUE)
    rice.yoasovi <- yoasovi_logitreg(categories = rice.train[,1], X = scale(rice.train[,-1]), learn_rate = rate_adagrad(1e-1), max_iter = 1000, patience=50, seed = seed, verbose = TRUE)

    return(rbind(
        cbind(Method = "BBVI", logitreg_summary(rice.bbvi)),
        cbind(Method = "QMCVI", logitreg_summary(rice.qmcvi)),
        cbind(Method = "YOASOVI", logitreg_summary(rice.yoasovi))
    ))
}

rice_performances <- do.call('rbind', rice_performances)
rice_performances <- rice_performances %>%
    group_by(Method) %>%
    summarise(iterations_mn = round(mean(iter),2), iterations_sd = round(sd(iter),2), time_mn = round(mean(elapsed),2), time_sd = round(sd(elapsed),2),
                elbo_mn = round(mean(elbo),2), elbo_sd = round(sd(elbo),2), DIC_mn = round(mean(DIC),2), DIC_sd = round(sd(DIC),2))

rice_performances

Algorithm converged after 59 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0029 | ELBO: -17902.66 | ELBO Change: 0

Algorithm converged after 147 iterations.

Algorithm converged after 66 iterations.

Algorithm converged after 83 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0023 | ELBO: -16638.62 | ELBO Change: 0

Algorithm converged after 153 iterations.

Algorithm converged after 66 iterations.

Algorithm converged after 41 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0026 | ELBO: -22004.58 | ELBO Change: 0

Algorithm converged after 182 iterations.

Algorithm converged after 99 iterations.

Algorithm converged after 46 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0025 | ELBO: -16607.05 | ELBO Change: 0

Algorithm converged after 194 iterations.

Algorithm converged after 92 iterations.

Algorithm converged after 64 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0035 | ELBO: -13777.99 | ELBO Change: 0

Algorithm converged after 165 iterations.

YOASOVI-Naiv

Method,iterations_mn,iterations_sd,time_mn,time_sd,elbo_mn,elbo_sd,DIC_mn,DIC_sd
<chr>,<dbl>,<dbl>,<drtn>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BBVI,54.6,23.60,6.17 secs,2.86,-3356.71,437.19,6750.66,874.7
QMCVI,164.1,20.55,2.52 secs,0.33,-11173.35,1597.66,22367.37,3194.3
YOASOVI,110.0,37.63,0.09 secs,0.05,-645.05,32.66,1323.48,65.3


## Diabetes

Also from the LibSVM collection: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary.html#diabetes

### Predictive Performance

In [24]:
diabetes <- read.table("Data/diabetes.csv", sep=" ", header = FALSE)
diabetes[,1] <- sapply(diabetes[[1]], function(x){
    if(x == -1) return(0)
    return(x)
})

index <- sample(x = nrow(diabetes), size = round(0.8 * nrow(diabetes)))

diabetes.train <- as.matrix(diabetes[index, ])
diabetes.test <- as.matrix(diabetes[-index, ])

In [25]:
diabetes.bbvi <- bbvi_logitreg(categories = diabetes.train[,1], X = scale(diabetes.train[,-1]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, verbose = TRUE)
diabetes.qmcvi <- qmcvi_logitreg(categories = diabetes.train[,1], X = scale(diabetes.train[,-1]), mc_size = 10, learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, verbose = TRUE)
diabetes.yoasovi <- yoasovi_logitreg(categories = diabetes.train[,1], X = scale(diabetes.train[,-1]), learn_rate = rate_adagrad(1e-1), max_iter = 1000, patience = 50, verbose = TRUE)

BBVI-JS+: Iteration 100 | lambda: 0.0049 | ELBO: -3467.76 | ELBO Change: 0

BBVI-JS+: Iteration 200 | lambda: 0.0041 | ELBO: -3599.09 | ELBO Change: 0

BBVI-JS+: Iteration 300 | lambda: 0.0032 | ELBO: -3626.58 | ELBO Change: 0

BBVI-JS+: Iteration 400 | lambda: 0.0029 | ELBO: -3512.94 | ELBO Change: 0

BBVI-JS+: Iteration 500 | lambda: 0.0036 | ELBO: -3635 | ELBO Change: 0

BBVI-JS+: Iteration 600 | lambda: 0.0021 | ELBO: -3558.93 | ELBO Change: 0

BBVI-JS+: Iteration 700 | lambda: 0.0014 | ELBO: -3652.67 | ELBO Change: 0

BBVI-JS+: Iteration 800 | lambda: 0.002 | ELBO: -3705.22 | ELBO Change: 0

BBVI-JS+: Iteration 900 | lambda: 0.002 | ELBO: -3500.31 | ELBO Change: 0

Algorithm converged after 927 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0022 | ELBO: -3410.73 | ELBO Change: 0

BBVI-Naive: Iteration 200 | lambda: 0.0017 | ELBO: -3806.25 | ELBO Change: 0

Algorithm converged after 225 iterations.

Algorithm converged after 55 iterations.



In [26]:
diabetes.bbvi.acc <- logitreg_accuracy(diabetes.bbvi, newdata = list(categories=diabetes.test[,1], X=scale(diabetes.test[,-1])))
diabetes.qmcvi.acc <- logitreg_accuracy(diabetes.qmcvi, newdata = list(categories=diabetes.test[,1], X=scale(diabetes.test[,-1])))
diabetes.yoasovi.acc <- logitreg_accuracy(diabetes.yoasovi, newdata = list(categories=diabetes.test[,1], X=scale(diabetes.test[,-1])))

In [27]:
diabetes_accuracy <- round(100 * rbind(
    diabetes.bbvi.acc$auc_simulations,
    diabetes.qmcvi.acc$auc_simulations,
    diabetes.yoasovi.acc$auc_simulations
),2)

diabetes_accuracy

mean,sd
60.28,8.55
60.36,8.58
65.31,9.73


### Fit Performance

In [28]:
diabetes_performances <- foreach(seed = 0:9) %do%{
    diabetes.bbvi <- bbvi_logitreg(categories = diabetes.train[,1], X = scale(diabetes.train[,-1]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    diabetes.qmcvi <- qmcvi_logitreg(categories = diabetes.train[,1], X = scale(diabetes.train[,-1]), mc_size = 10, learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    diabetes.yoasovi <- yoasovi_logitreg(categories = diabetes.train[,1], X = scale(diabetes.train[,-1]), learn_rate = rate_adagrad(1e-1), max_iter = 1000, patience = 50, seed = seed, verbose = TRUE)

    return(rbind(
        cbind(Method = "BBVI", logitreg_summary(diabetes.bbvi)),
        cbind(Method = "QMCVI", logitreg_summary(diabetes.qmcvi)),
        cbind(Method = "YOASOVI", logitreg_summary(diabetes.yoasovi))
    ))
}

diabetes_performances <- do.call('rbind', diabetes_performances)
diabetes_performances <- diabetes_performances %>%
    group_by(Method) %>%
    summarise(iterations_mn = round(mean(iter),2), iterations_sd = round(sd(iter),2), time_mn = round(mean(elapsed),2), time_sd = round(sd(elapsed),2),
                elbo_mn = round(mean(elbo),2), elbo_sd = round(sd(elbo),2), DIC_mn = round(mean(DIC),2), DIC_sd = round(sd(DIC),2))

diabetes_performances

BBVI-JS+: Iteration 100 | lambda: 0.0056 | ELBO: -3662.88 | ELBO Change: 0

BBVI-JS+: Iteration 200 | lambda: 0.0028 | ELBO: -3616.36 | ELBO Change: 0

BBVI-JS+: Iteration 300 | lambda: 0.003 | ELBO: -3621.04 | ELBO Change: 0

BBVI-JS+: Iteration 400 | lambda: 0.0026 | ELBO: -3559.43 | ELBO Change: 0

BBVI-JS+: Iteration 500 | lambda: 0.0016 | ELBO: -3525.19 | ELBO Change: 0

BBVI-JS+: Iteration 600 | lambda: 0.0024 | ELBO: -3634.46 | ELBO Change: 0

BBVI-JS+: Iteration 700 | lambda: 0.0022 | ELBO: -3544.72 | ELBO Change: 0

BBVI-JS+: Iteration 800 | lambda: 0.0016 | ELBO: -3660.36 | ELBO Change: 0

BBVI-JS+: Iteration 900 | lambda: 0.0015 | ELBO: -3555 | ELBO Change: 0

Algorithm converged after 924 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0024 | ELBO: -3509.35 | ELBO Change: 0

Algorithm converged after 165 iterations.

YOASOVI-Naive: Iteration 100 | ELBO: -405.54

Algorithm converged after 131 iterations.

BBVI-JS+: Iteration 100 | lambda: 0.0048 | ELBO: -3596.37 | ELBO Ch

Method,iterations_mn,iterations_sd,time_mn,time_sd,elbo_mn,elbo_sd,DIC_mn,DIC_sd
<chr>,<dbl>,<dbl>,<drtn>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BBVI,880.2,104.82,109.94 secs,36.59,-3552.64,76.61,7090.09,152.86
QMCVI,235.7,34.90,4.66 secs,2.15,-3319.70,274.17,6660.00,548.62
YOASOVI,94.8,32.14,0.08 secs,0.06,-491.62,61.88,1020.36,123.55


## Covertype

From the UCI Machine Learning repositiory: https://archive.ics.uci.edu/dataset/31/covertype

In [29]:
covertype <- read.table(gzfile("Data/covtype.data.gz"), sep = ",")
covertype <- as.matrix(covertype[covertype$V55 %in% c(1,2), ])

set.seed(923)
index <- sample(x = nrow(covertype), size = round(0.8 * nrow(covertype)))

covertype.train <- as.matrix(covertype[index, ])
covertype.test <- as.matrix(covertype[-index, ])
ind_dep <- ncol(covertype)

In [30]:
covertype.bbvi <- bbvi_logitreg(categories = covertype.train[,ind_dep], X = scale(covertype.train[,1:10]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, verbose = TRUE)
covertype.qmcvi <- qmcvi_logitreg(categories = covertype.train[,ind_dep], X = scale(covertype.train[,1:10]), mc_size = 10, learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, verbose = TRUE)
covertype.yoasovi <- yoasovi_logitreg(categories = covertype.train[,ind_dep], X = scale(covertype.train[,1:10]), learn_rate = rate_adagrad(1e-1), max_iter = 1000, patience = 50, verbose = TRUE)

BBVI-JS+: Iteration 100 | lambda: 0.0044 | ELBO: -2328803.28 | ELBO Change: 0

BBVI-JS+: Iteration 200 | lambda: 0.0029 | ELBO: -2191548.41 | ELBO Change: 0

BBVI-JS+: Iteration 300 | lambda: 0.0035 | ELBO: -2116106.62 | ELBO Change: 0

BBVI-JS+: Iteration 400 | lambda: 0.0023 | ELBO: -2281071.99 | ELBO Change: 0

BBVI-JS+: Iteration 500 | lambda: 0.002 | ELBO: -2314611.12 | ELBO Change: 0

BBVI-JS+: Iteration 600 | lambda: 0.0021 | ELBO: -2270067.12 | ELBO Change: 0

BBVI-JS+: Iteration 700 | lambda: 0.0018 | ELBO: -2373673.5 | ELBO Change: 0

BBVI-JS+: Iteration 800 | lambda: 0.002 | ELBO: -2207064.58 | ELBO Change: 0

BBVI-JS+: Iteration 900 | lambda: 0.0019 | ELBO: -2341831.99 | ELBO Change: 0

BBVI-JS+: Iteration 1000 | lambda: 0.0015 | ELBO: -2241135.47 | ELBO Change: 0

Algorithm did not converge after 1000 steps. Results may not be reliable.

BBVI-Naive: Iteration 100 | lambda: 0.0024 | ELBO: -2301304.46 | ELBO Change: 0

BBVI-Naive: Iteration 200 | lambda: 0.0015 | ELBO: -2256

In [31]:
covertype.bbvi.acc <- logitreg_accuracy(covertype.bbvi, newdata = list(categories=covertype.test[,ind_dep], X=scale(covertype.test[,1:10])))
covertype.qmcvi.acc <- logitreg_accuracy(covertype.qmcvi, newdata = list(categories=covertype.test[,ind_dep], X=scale(covertype.test[,1:10])))
covertype.yoasovi.acc <- logitreg_accuracy(covertype.yoasovi, newdata = list(categories=covertype.test[,ind_dep], X=scale(covertype.test[,1:10])))

In [32]:
covertype_accuracy <- round(100 * rbind(
    covertype.bbvi.acc$auc_simulations,
    covertype.qmcvi.acc$auc_simulations,
    covertype.yoasovi.acc$auc_simulations
),2)

covertype_accuracy

mean,sd
52.18,6.66
52.18,6.68
57.74,6.45


### Fit Performance

In [33]:
covertype_performances <- foreach(seed = 0:9) %do%{
    covertype.bbvi <- bbvi_logitreg(categories = covertype.train[,ind_dep], X = scale(covertype.train[,1:10]), mc_size = 100, method = "JS+", learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    covertype.qmcvi <- qmcvi_logitreg(categories = covertype.train[,ind_dep], X = scale(covertype.train[,1:10]), mc_size = 10, learn_rate = rate_adagrad(1e-1), max_iter = 1000, converge = 1e-3, seed = seed, verbose = TRUE)
    covertype.yoasovi <- yoasovi_logitreg(categories = covertype.train[,ind_dep], X = scale(covertype.train[,1:10]), learn_rate = rate_adagrad(1e-1), max_iter = 1000, patience = 50, seed = seed, verbose = TRUE)

    return(rbind(
        cbind(Method = "BBVI", logitreg_summary(covertype.bbvi)),
        cbind(Method = "QMCVI", logitreg_summary(covertype.qmcvi)),
        cbind(Method = "YOASOVI", logitreg_summary(covertype.yoasovi))
    ))
}

covertype_performances <- do.call('rbind', covertype_performances)
covertype_performances <- covertype_performances %>%
    group_by(Method) %>%
    summarise(iterations_mn = round(mean(iter),2), iterations_sd = round(sd(iter),2), time_mn = round(mean(elapsed),2), time_sd = round(sd(elapsed),2),
                elbo_mn = round(mean(elbo),2), elbo_sd = round(sd(elbo),2), DIC_mn = round(mean(DIC),2), DIC_sd = round(sd(DIC),2))

covertype_performances

BBVI-JS+: Iteration 100 | lambda: 0.0041 | ELBO: -2181530.38 | ELBO Change: 0

BBVI-JS+: Iteration 200 | lambda: 0.0028 | ELBO: -2266908.02 | ELBO Change: 0

BBVI-JS+: Iteration 300 | lambda: 0.0028 | ELBO: -2322004.08 | ELBO Change: 0

BBVI-JS+: Iteration 400 | lambda: 0.0025 | ELBO: -2347581.93 | ELBO Change: 0

BBVI-JS+: Iteration 500 | lambda: 0.002 | ELBO: -2279457.54 | ELBO Change: 0

BBVI-JS+: Iteration 600 | lambda: 0.0018 | ELBO: -2171379.06 | ELBO Change: 0

BBVI-JS+: Iteration 700 | lambda: 0.0019 | ELBO: -2237103.65 | ELBO Change: 0

BBVI-JS+: Iteration 800 | lambda: 0.0017 | ELBO: -2188420.49 | ELBO Change: 0

BBVI-JS+: Iteration 900 | lambda: 0.0014 | ELBO: -2339342.26 | ELBO Change: 0

Algorithm converged after 977 iterations.

BBVI-Naive: Iteration 100 | lambda: 0.0022 | ELBO: -2340007.95 | ELBO Change: 0

Algorithm converged after 199 iterations.

Algorithm converged after 60 iterations.

BBVI-JS+: Iteration 100 | lambda: 0.0033 | ELBO: -2173963.67 | ELBO Change: 0

BB

Method,iterations_mn,iterations_sd,time_mn,time_sd,elbo_mn,elbo_sd,DIC_mn,DIC_sd
<chr>,<dbl>,<dbl>,<drtn>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BBVI,956.4,43.44,11013.19 secs,5809.21,-2265243.7,71403.32,4530468,142805.6
QMCVI,206.4,14.63,222.21 secs,72.92,-1787187.3,188529.91,3574402,377060.3
YOASOVI,101.3,44.22,4.84 secs,3.68,-363684.2,42045.17,727416,84093.7


In [ ]:
saveRDS(list("performance"=covertype_performances, "accuracy"=covertype_accuracy), 
        "Outputs/logit_covertype.RDS")

covertype_performances <- readRDS("Outputs/logit_covertype.RDS")$covertype_performances
covertype_accuracy <- readRDS("Outputs/logit_covertype.RDS")$covertype_accuracy

### Kable

In [35]:
all_performances <- rbind(
    cbind(Dataset = "Wine", wine_performances, wine_accuracy),
    cbind(Dataset = "Rice", rice_performances, rice_accuracy),
    cbind(Dataset = "Diabetes", diabetes_performances, diabetes_accuracy),
    cbind(Dataset = "Covertype", covertype_performances, covertype_accuracy)
)

all_performances %>%
    select(Dataset, Method, time_mn, time_sd, elbo_mn, elbo_sd, DIC_mn, DIC_sd, mean, sd) %>%
    kable('latex', booktabs = TRUE, col.names = c("Data","Method","Mean","SD","Mean","SD","Mean","SD","Mean","SD")) %>%
    add_header_above(c("", "", "Time (secs)" = 2, "ELBO" = 2, "DIC" = 2, "AUC" = 2)) %>%
    kable_styling(latex_options=c("repeat_header"), font_size  = 9) %>%
    print()

\begin{table}
\centering\begingroup\fontsize{9}{11}\selectfont

\begin{tabular}{lllrrrrrrr}
\toprule
\multicolumn{1}{c}{} & \multicolumn{1}{c}{} & \multicolumn{2}{c}{Time (secs)} & \multicolumn{2}{c}{ELBO} & \multicolumn{2}{c}{DIC} & \multicolumn{2}{c}{AUC} \\
\cmidrule(l{3pt}r{3pt}){3-4} \cmidrule(l{3pt}r{3pt}){5-6} \cmidrule(l{3pt}r{3pt}){7-8} \cmidrule(l{3pt}r{3pt}){9-10}
Data & Method & Mean & SD & Mean & SD & Mean & SD & Mean & SD\\
\midrule
Wine & BBVI & 3.55 secs & 1.56 & -95.59 & 21.34 & 255.05 & 42.73 & 84.71 & 14.92\\
Wine & QMCVI & 2.16 secs & 0.22 & -332.66 & 59.91 & 700.68 & 117.42 & 78.54 & 16.84\\
Wine & YOASOVI & 0.04 secs & 0.02 & -5.44 & 1.77 & 72.76 & 4.78 & 80.78 & 16.66\\
Rice & BBVI & 6.17 secs & 2.86 & -3356.71 & 437.19 & 6750.66 & 874.70 & 84.92 & 12.95\\
Rice & QMCVI & 2.52 secs & 0.33 & -11173.35 & 1597.66 & 22367.37 & 3194.30 & 74.71 & 15.35\\
\addlinespace
Rice & YOASOVI & 0.09 secs & 0.05 & -645.05 & 32.66 & 1323.48 & 65.30 & 81.01 & 14.40\\
Diabetes & BBVI